In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark_session = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql.functions import *

# Read Data

In [ ]:
items = spark_session.read.option("inferSchema","true").csv("drive/MyDrive/All/Data/items.csv", header=True, sep="|")

In [ ]:
items.show(3)

+------+--------------------+------------------+--------------------+----------+--------------------+
|itemID|               title|            author|           publisher|main topic|           subtopics|
+------+--------------------+------------------+--------------------+----------+--------------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|       YFB|               [5AH]|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|       AGZ|[5AJ,AGZ,WFA,YBG,...|
| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|       YFH|           [5AP,FBA]|
+------+--------------------+------------------+--------------------+----------+--------------------+
only showing top 3 rows



# Data Preprocessing

In [ ]:
items.schema

StructType(List(StructField(itemID,IntegerType,true),StructField(title,StringType,true),StructField(author,StringType,true),StructField(publisher,StringType,true),StructField(main topic,StringType,true),StructField(subtopics,StringType,true)))

In [ ]:
items = items.withColumn("subtopics",translate(items["subtopics"],"[",""))
items = items.withColumn("subtopics",translate(items["subtopics"],"]",""))

### Delete Item 62676

In [ ]:
# items.select([count(when(isnull('main topic'), True))]).show()
# items.select([count(when(isnull('subtopics'), True))]).show()
# items.select([count(when(isnull('subtopics') & isnull('main topic'), True))]).show()
# items.select([count(when(items["subtopics"] == "", True))]).show()

In [ ]:
# items.where((isnull('subtopics') & isnull('main topic'))).show()

In [ ]:
items.where("itemID = 62676").show()

+------+--------------------+------+---------+----------+---------+
|itemID|               title|author|publisher|main topic|subtopics|
+------+--------------------+------+---------+----------+---------+
| 62676|How To Be A Diva|...|  null|     null|      null|     null|
+------+--------------------+------+---------+----------+---------+



In [ ]:
items = items.filter(items.itemID != 62676)

In [ ]:
# items.where((isnull('subtopics') & isnull('main topic'))).show()

In [ ]:
# items.show(3)

### Combine Main Topic and Subtopics

In [ ]:
items = items.withColumn(
    "topics", 
    when(isnull("main topic"), items["subtopics"]).
    when(items["subtopics"] == "", items["main topic"]).
    otherwise(concat(col("main topic"), lit(","), col("subtopics")))
) 

In [ ]:
items.show(3)
items.where(isnull('main topic')).show(5)
items.where(items["subtopics"] == "").show(3)

+------+--------------------+------------------+--------------------+----------+--------------------+--------------------+
|itemID|               title|            author|           publisher|main topic|           subtopics|              topics|
+------+--------------------+------------------+--------------------+----------+--------------------+--------------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|       YFB|                 5AH|             YFB,5AH|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|       AGZ|5AJ,AGZ,WFA,YBG,Y...|AGZ,5AJ,AGZ,WFA,Y...|
| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|       YFH|             5AP,FBA|         YFH,5AP,FBA|
+------+--------------------+------------------+--------------------+----------+--------------------+--------------------+
only showing top 3 rows

+------+--------------------+--------------------+-------------------+----------+------------+------------+
|itemI

In [ ]:
items = items.select("itemID","title","author","publisher","topics")

In [ ]:
items.show(3)

+------+--------------------+------------------+--------------------+--------------------+
|itemID|               title|            author|           publisher|              topics|
+------+--------------------+------------------+--------------------+--------------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|             YFB,5AH|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|AGZ,5AJ,AGZ,WFA,Y...|
| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|         YFH,5AP,FBA|
+------+--------------------+------------------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
items = items.withColumn(
    'topics', array_distinct(split(col("topics"),","))
)

In [ ]:
items.select("topics").show(5, False)
items.show(3)

+-----------------------------------+
|topics                             |
+-----------------------------------+
|[YFB, 5AH]                         |
|[AGZ, 5AJ, WFA, YBG, YBL, YNA, YPA]|
|[YFH, 5AP, FBA]                    |
|[YB, 5AC, 5AD, YBG, YBL, YF]       |
|[WFTM, WD, YBG, YBL, YBLD, YBLN1]  |
+-----------------------------------+
only showing top 5 rows

+------+--------------------+------------------+--------------------+--------------------+
|itemID|               title|            author|           publisher|              topics|
+------+--------------------+------------------+--------------------+--------------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|          [YFB, 5AH]|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|[AGZ, 5AJ, WFA, Y...|
| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|     [YFH, 5AP, FBA]|
+------+--------------------+------------------+--------------------+------------------

In [ ]:
items = items.limit(100)

### Explode into separate rows

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
items = items.withColumn("topics_splitted", explode(items.topics))

In [ ]:
items.show(5)

+------+--------------------+------------------+--------------------+--------------------+---------------+
|itemID|               title|            author|           publisher|              topics|topics_splitted|
+------+--------------------+------------------+--------------------+--------------------+---------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|          [YFB, 5AH]|            YFB|
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|          [YFB, 5AH]|            5AH|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|[AGZ, 5AJ, WFA, Y...|            AGZ|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|[AGZ, 5AJ, WFA, Y...|            5AJ|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|[AGZ, 5AJ, WFA, Y...|            WFA|
+------+--------------------+------------------+--------------------+--------------------+---------------+
only showing top 5 rows



In [ ]:
items = items.drop("topics")

In [ ]:
items.show(3)

+------+--------------------+------------------+--------------------+---------------+
|itemID|               title|            author|           publisher|topics_splitted|
+------+--------------------+------------------+--------------------+---------------+
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|            YFB|
| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|            5AH|
| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|            AGZ|
+------+--------------------+------------------+--------------------+---------------+
only showing top 3 rows



### Pivot

In [ ]:
from pyspark.sql.functions import sum

In [ ]:
spark_session.conf.set("spark.sql.pivotMaxValues",25000)

In [ ]:
pivot = items.groupBy("itemID").pivot("topics_splitted").count()

In [ ]:
pivot = pivot.fillna(0)

In [ ]:
pivot.count()

100

In [ ]:
pivot.show()

+------+----------+----+---+----+------------+-----------+-----+----+----+---+-----+-----+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+---+----+---+----+---+----+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+----+----+---+---+----+----+---+---+----+---+---+---+---+----+---+----+---+----+----+-----+---+---+---+---+---+----+----+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+-----+----+---+----+----+---+----+----+-----+----+---+---+---+---+---+----+----+---+---+---+
|itemID|1DFG-DE-VH|1DND| 1F|1KBB|1KBB-US-MLCC|1KBB-US-NAK|1KLSC|2ACG|3KLN|3MR|3MRBA|3MRBF|4CA|4CD|5ABK|5AC|5AD|5AF|5AG|5AH|5AJ|5AK|5AL|5AM|5AN|5AP|5AQ|5AX|5HKB|5HPD|5HPF|5JA|5LKE|5PG|5PSG|AGZ|ATDH|ATF|ATFN|DND|DSY| FB|FBA|FDB|FFJ| FJ| FL| FM|FMB|FMH|FMM|FMR|FMW|FMX| FN| FP| FR|FRD|FRM|FRT|FRX|FXB|FYB|FYR|FYT|FYV| FZ|JBSF1|JNLA|SFBC|SXE| WD|WDHW|WDKX|WDM|WFA|WFTM|WFU|WZG|WZS| Y

### Dimensionality Reduction

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [ ]:
mat = pivot.drop("itemID").rdd.map(lambda s : Vectors.dense(s))

In [ ]:
mat = RowMatrix(mat)

### Principal component analysis (PCA)

In [ ]:
pca = mat.computePrincipalComponents(5)

In [ ]:
projected = mat.multiply(pca)

In [ ]:
print(projected.rows.collect())

[DenseVector([-0.0802, 0.0592, 0.2229, 0.1696, 0.1981]), DenseVector([1.3049, 0.1207, -0.1314, -0.1071, -0.0875]), DenseVector([-0.2068, -0.2694, 0.5398, -0.0766, -0.4487]), DenseVector([1.494, 0.1544, -0.3914, -0.1487, -0.294]), DenseVector([1.2363, 0.1364, -0.3026, -0.1786, -0.4839]), DenseVector([-0.2432, 0.0916, -0.3232, 1.0741, -0.4166]), DenseVector([0.6743, 0.0779, -0.2221, -0.1626, -0.8842]), DenseVector([-0.5368, 0.2606, 0.543, -0.8693, -0.4141]), DenseVector([-0.5368, 0.2606, 0.543, -0.8693, -0.4141]), DenseVector([-0.08, 0.0703, 0.1955, 0.222, 0.2868]), DenseVector([1.2872, 0.309, 0.3233, -0.4873, -0.7836]), DenseVector([-0.3233, 0.3693, -0.472, 0.8999, -0.4877]), DenseVector([-0.2094, -0.3864, -0.2701, 0.1712, -0.1179]), DenseVector([0.7339, 0.0473, 0.0247, 0.028, 0.4532]), DenseVector([0.7339, 0.0473, 0.0247, 0.028, 0.4532]), DenseVector([0.9461, 0.0729, 0.0644, 0.0242, 0.7228]), DenseVector([1.3458, 0.1728, -0.2791, -0.223, -0.3317]), DenseVector([-0.3581, 1.1736, -0.2608

In [ ]:
projected.numRows()

100

In [ ]:
projected.numCols()

5

### Convert to pandas dataframe

In [ ]:
import pandas as pd

In [ ]:
training = projected.rows.map(lambda x: x.toArray().tolist()).collect()

In [ ]:
data = pd.Series(training, pd.MultiIndex.from_product([[x for x in range(100)], list('A')])).unstack()

In [ ]:
data = data.rename(columns = {'A': 'Features'}, inplace = False)

In [ ]:
data

,Features
0,"[-0.08023468995861298, 0.059166158116907605, 0..."
1,"[1.3049456623192097, 0.12070246816510052, -0.1..."
2,"[-0.20675283197411776, -0.26935805525868395, 0..."
3,"[1.4940414871086798, 0.15444557748503157, -0.3..."
4,"[1.2363475143441267, 0.1363626029379749, -0.30..."
...,...
95,"[-0.1289781001618532, 0.08493979509130009, 0.4..."
96,"[-0.35257365380523054, -0.46104651528811497, -..."
97,"[-1.0117151612658464, 1.641298330680523, -0.85..."
98,"[-0.26226922408244846, 0.5444702654043473, -0...."


In [ ]:
item_pd = items.limit(100).toPandas()

In [ ]:
data["itemID"] = item_pd["itemID"]

In [ ]:
data

,Features,itemID,d0,d1,d2,d3,d4,d5,i0,i1,i2,i3,i4,i5
0,"[-0.08023468995861298, 0.059166158116907605, 0...",21310,0.000000e+00,2.245785e-02,3.806018e-02,0.053646,0.062399,0.290401,0,9,95,94,77,38
1,"[1.3049456623192097, 0.12070246816510052, -0.1...",21310,0.000000e+00,6.630074e-04,6.344000e-03,0.008302,0.019039,0.020403,1,81,82,33,3,86
2,"[-0.20675283197411776, -0.26935805525868395, 0...",73018,0.000000e+00,4.794997e-02,4.913574e-02,0.049960,0.055286,0.055527,2,91,42,93,49,44
3,"[1.4940414871086798, 0.15444557748503157, -0.3...",73018,1.110223e-16,1.850701e-03,1.850701e-03,0.001851,0.001941,0.002994,3,87,83,86,80,33
4,"[1.2363475143441267, 0.1363626029379749, -0.30...",73018,1.110223e-16,6.809075e-04,1.407070e-03,0.001991,0.008846,0.008980,4,53,84,85,16,80
5,"[-0.24322621025234634, 0.09161779842624597, -0...",73018,0.000000e+00,0.000000e+00,4.684993e-02,0.083550,0.113169,0.186997,5,52,11,88,19,20
6,"[0.6742919231799532, 0.07787372724985975, -0.2...",73018,0.000000e+00,7.226263e-06,7.291772e-04,0.109403,0.111102,0.119425,6,28,29,85,84,53
7,"[-0.5368141208782047, 0.2605632356222075, 0.54...",73018,0.000000e+00,0.000000e+00,1.060873e-01,0.117307,0.280811,0.330343,7,8,59,99,51,40
8,"[-0.5368141208782047, 0.2605632356222075, 0.54...",73018,0.000000e+00,0.000000e+00,1.060873e-01,0.117307,0.280811,0.330343,7,8,59,99,51,40
9,"[-0.08000345455598978, 0.0703310996026805, 0.1...",19194,0.000000e+00,2.245785e-02,3.897150e-02,0.042994,0.128398,0.218122,9,0,95,77,94,38


# Model

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nearest_neighbors = 6 

#### Knn, Cosine, Brute, p=2 (euclidean distance)

In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', p=2)

In [ ]:
evaluation = pd.read_csv('drive/MyDrive/All/Data/evaluation.csv', sep='|')

In [ ]:
evaluation_items = pd.concat([evaluation, data])

In [ ]:
evaluation_items

,itemID,Features
0,12,NaN
1,45274,NaN
2,10104,NaN
3,41371,NaN
4,14015,NaN
...,...,...
95,11117,"[-0.1289781001618532, 0.08493979509130009, 0.4..."
96,11117,"[-0.35257365380523054, -0.46104651528811497, -..."
97,11117,"[-1.0117151612658464, 1.641298330680523, -0.85..."
98,73865,"[-0.26226922408244846, 0.5444702654043473, -0...."


##### Binary

In [ ]:
result_items = data

In [ ]:
knn.fit(training)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
distances, indices = knn.kneighbors(training, n_neighbors=nearest_neighbors)

In [ ]:
distances_items = pd.DataFrame(distances)

In [ ]:
result_items["d0"] = distances_items[0]
result_items["d1"] = distances_items[1]
result_items["d2"] = distances_items[2]
result_items["d3"] = distances_items[3]
result_items["d4"] = distances_items[4]
result_items["d5"] = distances_items[5]

In [ ]:
indices_items = pd.DataFrame(indices)

In [ ]:
result_items["i0"] = indices_items[0]
result_items["i1"] = indices_items[1]
result_items["i2"] = indices_items[2]
result_items["i3"] = indices_items[3]
result_items["i4"] = indices_items[4]
result_items["i5"] = indices_items[5]

In [ ]:
pd.set_option("display.max_rows",None)
result_items

,Features,itemID,d0,d1,d2,d3,d4,d5,i0,i1,i2,i3,i4,i5
0,"[-0.08023468995861298, 0.059166158116907605, 0...",21310,0.000000e+00,2.245785e-02,3.806018e-02,0.053646,0.062399,0.290401,0,9,95,94,77,38
1,"[1.3049456623192097, 0.12070246816510052, -0.1...",21310,0.000000e+00,6.630074e-04,6.344000e-03,0.008302,0.019039,0.020403,1,81,82,33,3,86
2,"[-0.20675283197411776, -0.26935805525868395, 0...",73018,0.000000e+00,4.794997e-02,4.913574e-02,0.049960,0.055286,0.055527,2,91,42,93,49,44
3,"[1.4940414871086798, 0.15444557748503157, -0.3...",73018,1.110223e-16,1.850701e-03,1.850701e-03,0.001851,0.001941,0.002994,3,87,83,86,80,33
4,"[1.2363475143441267, 0.1363626029379749, -0.30...",73018,1.110223e-16,6.809075e-04,1.407070e-03,0.001991,0.008846,0.008980,4,53,84,85,16,80
5,"[-0.24322621025234634, 0.09161779842624597, -0...",73018,0.000000e+00,0.000000e+00,4.684993e-02,0.083550,0.113169,0.186997,5,52,11,88,19,20
6,"[0.6742919231799532, 0.07787372724985975, -0.2...",73018,0.000000e+00,7.226263e-06,7.291772e-04,0.109403,0.111102,0.119425,6,28,29,85,84,53
7,"[-0.5368141208782047, 0.2605632356222075, 0.54...",73018,0.000000e+00,0.000000e+00,1.060873e-01,0.117307,0.280811,0.330343,7,8,59,99,51,40
8,"[-0.5368141208782047, 0.2605632356222075, 0.54...",73018,0.000000e+00,0.000000e+00,1.060873e-01,0.117307,0.280811,0.330343,7,8,59,99,51,40
9,"[-0.08000345455598978, 0.0703310996026805, 0.1...",19194,0.000000e+00,2.245785e-02,3.897150e-02,0.042994,0.128398,0.218122,9,0,95,77,94,38


In [ ]:
for i, row in result_items.iterrows():
  recommendation_count = 1
  for j in range(6):
    if recommendation_count == 6:
      break
    
    indices_name = "i" + str(j)
    column_name = "rec_" + str(recommendation_count)
    recommended_item_id = result_items.at[result_items.at[i, indices_name], "itemID"]
    if recommended_item_id == result_items.at[i, "itemID"]:
      continue
    else:
      result_items.at[i,column_name] = recommended_item_id
      recommendation_count = recommendation_count + 1

In [ ]:
result_items["rec_1"] = result_items["rec_1"].astype('int64')
result_items["rec_2"] = result_items["rec_2"].astype('int64')
result_items["rec_3"] = result_items["rec_3"].astype('int64')
result_items["rec_4"] = result_items["rec_4"].astype('int64')
result_items["rec_5"] = result_items["rec_5"].astype('int64')

ValueError: ignored

In [ ]:
result_items[["itemID","rec_1","rec_2","rec_3","rec_4","rec_5"]]

,itemID,rec_1,rec_2,rec_3,rec_4,rec_5
0,21310,19194,11117,11117.0,62433.0,17731.0
1,21310,72603,72603,14576.0,73018.0,72603.0
2,73018,11989,58723,11117.0,33976.0,73124.0
3,73018,72603,72603,72603.0,72603.0,14576.0
4,73018,2408,72603,72603.0,40250.0,72603.0
5,73018,33976,19194,11989.0,46107.0,46107.0
6,73018,34217,31436,72603.0,72603.0,2408.0
7,73018,29508,73865,33976.0,58723.0,NaN
8,73018,29508,73865,33976.0,58723.0,NaN
9,19194,21310,11117,62433.0,11117.0,17731.0


Apply to Evaluation

In [ ]:
evaluation_items = evaluation_items.dropna()

In [ ]:
evaluation_items

,itemID,Features
0,21310,"[-0.08023468995861298, 0.059166158116907605, 0..."
1,21310,"[1.3049456623192097, 0.12070246816510052, -0.1..."
2,73018,"[-0.20675283197411776, -0.26935805525868395, 0..."
3,73018,"[1.4940414871086798, 0.15444557748503157, -0.3..."
4,73018,"[1.2363475143441267, 0.1363626029379749, -0.30..."
5,73018,"[-0.24322621025234634, 0.09161779842624597, -0..."
6,73018,"[0.6742919231799532, 0.07787372724985975, -0.2..."
7,73018,"[-0.5368141208782047, 0.2605632356222075, 0.54..."
8,73018,"[-0.5368141208782047, 0.2605632356222075, 0.54..."
9,19194,"[-0.08000345455598978, 0.0703310996026805, 0.1..."


In [ ]:
distances, indices = knn.kneighbors(evaluation_items["Features"].tolist(), n_neighbors=nearest_neighbors)

In [ ]:
distances_items_evaluation = pd.DataFrame(distances)

In [ ]:
evaluation_items["d0"] = distances_items_evaluation[0]
evaluation_items["d1"] = distances_items_evaluation[1]
evaluation_items["d2"] = distances_items_evaluation[2]
evaluation_items["d3"] = distances_items_evaluation[3]
evaluation_items["d4"] = distances_items_evaluation[4]
evaluation_items["d5"] = distances_items_evaluation[5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
indices_items_evaluation = pd.DataFrame(indices)

In [ ]:
evaluation_items["i0"] = indices_items_evaluation[0]
evaluation_items["i1"] = indices_items_evaluation[1]
evaluation_items["i2"] = indices_items_evaluation[2]
evaluation_items["i3"] = indices_items_evaluation[3]
evaluation_items["i4"] = indices_items_evaluation[4]
evaluation_items["i5"] = indices_items_evaluation[5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
for i, row in evaluation_items.iterrows():
  recommendation_count = 1
  for j in range(6):
    if recommendation_count == 6:
      break
    
    indices_name = "i" + str(j)
    column_name = "rec_" + str(recommendation_count)
    recommended_item_id = evaluation_items.at[evaluation_items.at[i, indices_name], "itemID"]
    if recommended_item_id == evaluation_items.at[i, "itemID"]:
      continue
    else:
      evaluation_items.at[i,column_name] = recommended_item_id
      recommendation_count = recommendation_count + 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
evaluation_items["rec_1"] = evaluation_items["rec_1"].astype('int64')
evaluation_items["rec_2"] = evaluation_items["rec_2"].astype('int64')
evaluation_items["rec_3"] = evaluation_items["rec_3"].astype('int64')
evaluation_items["rec_4"] = evaluation_items["rec_4"].astype('int64')
evaluation_items["rec_5"] = evaluation_items["rec_5"].astype('int64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ValueError: ignored

In [ ]:
evaluation_items[["itemID","rec_1","rec_2","rec_3","rec_4","rec_5"]]

,itemID,rec_1,rec_2,rec_3,rec_4,rec_5
0,21310,19194,11117,11117.0,62433.0,17731.0
1,21310,72603,72603,14576.0,73018.0,72603.0
2,73018,11989,58723,11117.0,33976.0,73124.0
3,73018,72603,72603,72603.0,72603.0,14576.0
4,73018,2408,72603,72603.0,40250.0,72603.0
5,73018,33976,19194,11989.0,46107.0,46107.0
6,73018,34217,31436,72603.0,72603.0,2408.0
7,73018,29508,73865,33976.0,58723.0,NaN
8,73018,29508,73865,33976.0,58723.0,NaN
9,19194,21310,11117,62433.0,11117.0,17731.0
